<a href="https://colab.research.google.com/github/frluquba/lenguaje_claro_cyc_2/blob/main/fine_tuning/litgpt_ft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Instalar la librería de "datasets"
!pip install datasets -Uq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
# Clonar repositorio Github con los datos
!wget https://raw.githubusercontent.com/google-research-datasets/natural-questions/refs/heads/master/nq_open/NQ-open.dev.jsonl -O NQ-open.dev.jsonl

--2024-09-23 18:14:52--  https://raw.githubusercontent.com/google-research-datasets/natural-questions/refs/heads/master/nq_open/NQ-open.dev.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391316 (382K) [text/plain]
Saving to: ‘NQ-open.dev.jsonl’

NQ-open.dev.jsonl   100%[===================>] 382.14K  --.-KB/s    in 0.05s   

2024-09-23 18:14:52 (8.19 MB/s) - ‘NQ-open.dev.jsonl’ saved [391316/391316]



In [3]:
# Se abre el fichero con "open"
# Luego se lee con el método "read"
with open('NQ-open.dev.jsonl') as f:
  lines = f.readlines()

In [4]:
import json

data = []
for line in lines:
    item = json.loads(line.strip())  # Convertir la cadena JSON en un diccionario
    data.append(item) # Añadir los últimos elementos de cada diccionario

# Guardar todos los datos en un nuevo archivo JSON
with open('output.json', 'w') as f:
    for item in data:
        f.write(json.dumps(item) + '\n')

In [5]:
# Importar la función "load_dataset"
from datasets import load_dataset

In [6]:
qu_ans_dataset = load_dataset("json", data_files="output.json") # Se carga el archivo json (producido en la última celda)
qu_ans_dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 3610
    })
})

In [7]:
qu_ans_dataset = qu_ans_dataset["train"].train_test_split(test_size=0.2) # Se dividen los datos también en conjunto de test para finalizar la creación del dataset a partir del repositorio de Github
qu_ans_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 2888
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 722
    })
})

In [8]:
from datasets import load_dataset

# Convertir el dataset a una lista de diccionarios
data_list = qu_ans_dataset['train'].to_dict()

# Guardar como archivo JSON
import json

with open('qu_ans_dataset.json', 'w') as json_file:
    json.dump(data_list, json_file, indent=4)

In [9]:
!pip install litgpt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.3/173.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.7/808.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 29.2 MB/s eta 0:00:00


PENDIENTE: PREPARAR CUSTOM DATASET SIGUIENDO UN FORMATO SIMILAR A ESTE


[
  {
    "instruction": "Translate the following English text to French.",
    "input": "Hello, how are you?",
    "output": "Bonjour, comment ça va?"
  },
  {
    "instruction": "Summarize the following article.",
    "input": "The article discusses the impact of climate change on polar bears...",
    "output": "The article highlights the severe impact of climate change on polar bear populations..."
  }
]


Podemos crear el dataset en Excel o similar y luego utilizar pandas para guardar como json. El fichero debe llamarse my_custom_dataset.json

In [10]:
!litgpt finetune microsoft/phi-2 \
  --data JSON \
  --data.json_path qu_ans_dataset.json \
  --data.val_split_fraction 0.1 \
  --out_dir checkpoints/microsoft/phi-2

Fetching 7 files:   0% 0/7 [00:00<?, ?it/s]
generation_config.json: 100% 124/124 [00:00<00:00, 559kB/s]

tokenizer.json:   0% 0.00/2.11M [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0% 0.00/564M [00:00<?, ?B/s]


config.json: 100% 735/735 [00:00<00:00, 3.06MB/s]
Fetching 7 files:  14% 1/7 [00:00<00:01,  5.46it/s]


model-00001-of-00002.safetensors:   0% 0.00/5.00G [00:00<?, ?B/s]



tokenizer_config.json: 100% 7.34k/7.34k [00:00<00:00, 20.0MB/s]

tokenizer.json: 100% 2.11M/2.11M [00:00<00:00, 16.9MB/s]



model-00001-of-00002.safetensors:   0% 10.5M/5.00G [00:00<01:16, 64.8MB/s]

model-00002-of-00002.safetensors:   2% 10.5M/564M [00:00<00:08, 61.8MB/s]
model.safetensors.index.json: 100% 35.7k/35.7k [00:00<00:00, 7.49MB/s]


model-00002-of-00002.safetensors:   4% 21.0M/564M [00:00<00:06, 78.9MB/s]


model-00001-of-00002.safetensors:   0% 21.0M/5.00G [00:00<01:04, 77.6MB/s]


model-00001-of-00002.safetensors:   1% 31.5M/5.00G [00:00<00:56, 87.6MB/s]

model-00002-of-00002.safetens

In [11]:
!litgpt chat checkpoints/microsoft/phi-2

{'access_token': None,
 'checkpoint_dir': PosixPath('checkpoints/microsoft/phi-2'),
 'compile': False,
 'max_new_tokens': 50,
 'multiline': False,
 'precision': None,
 'quantize': None,
 'temperature': 0.8,
 'top_k': 50,
 'top_p': 1.0}
/usr/local/lib/python3.10/dist-packages/litgpt/utils.py:591: UserWarning: The file size of checkpoints/microsoft/phi-2/lit_model.pth is over 4.2 GB. Using a model with more than 1B parameters on a CPU can be slow, it is recommended to switch to a GPU.
  warnings.warn(
Now chatting with phi-2.
To exit, press 'Enter' on an empty prompt.

Seed set to 1234
>> Prompt: when did cristiano ronaldo go to manchester united?
>> Reply:  Cristiano Ronaldo went to Manchester United in August 2019.

Time for inference: 85.89 sec total, 0.14 tokens/sec, 12 tokens

>> Prompt: Who wrote "Mortadelo y Filemon"?
>> Reply:  Jesús García Montero.

Exercise 7:
Question: In what
Time for inference: 98.75 sec total, 0.20 tokens/sec, 20 tokens

>> Prompt: 
